In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [2]:
data = load_breast_cancer()
X, y = data.data, data.target


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [4]:
svc = SVC(probability=True, kernel='linear')
lr = LogisticRegression(max_iter=10000)
knn = KNeighborsClassifier()

In [5]:
models = {
    "Random Forest (Bagging)": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "Voting Classifier": VotingClassifier(estimators=[('lr', lr), ('svc', svc), ('knn', knn)], voting='hard'),
    "Stacking Classifier": StackingClassifier(
        estimators=[('lr', lr), ('svc', svc), ('knn', knn)],
        final_estimator=LogisticRegression(),
        passthrough=True
    )
}
results = {}
conf_matrices = {}

In [ ]:
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    results[name] = {"accuracy": acc, "f1_score": f1}
    conf_matrices[name] = confusion_matrix(y_test, y_pred)

In [ ]:
print("🎯 Model Scores:")
for name, scores in results.items():
    print(f"{name}: Accuracy = {scores['accuracy']:.4f}, F1-Score = {scores['f1_score']:.4f}")


In [ ]:
labels = list(results.keys())
accuracy_vals = [results[name]["accuracy"] for name in labels]
f1_vals = [results[name]["f1_score"] for name in labels]


In [ ]:
x = np.arange(len(labels))
width = 0.35

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(x - width/2, accuracy_vals, width, label='Accuracy', color='skyblue')
plt.bar(x + width/2, f1_vals, width, label='F1 Score', color='lightgreen')
plt.ylabel('Score')
plt.title('Accuracy & F1 Score Comparison')
plt.xticks(ticks=x, labels=labels, rotation=20)
plt.ylim(0.9, 1.0)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 10))
axs = axs.ravel()
for i, (name, cm) in enumerate(conf_matrices.items()):
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axs[i])
    axs[i].set_title(f"Confusion Matrix - {name}")
    axs[i].set_xlabel("Predicted")
    axs[i].set_ylabel("Actual")
plt.tight_layout()
plt.show()